In [ ]:
%%capture
%pip install pandas matplotlib plotly 'nbformat>=4.2.0' 'ipywidgets>=7.6'
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

from quisp import (
    QNode,
    Network,
    ChannelOption,
    QNodeAddr,
    NativeSimulator,
    Config,
    NetworkBuilder,
    LinkType,
)

from analysis_utils import count_rulesets_by_responders

In [ ]:
n = NetworkBuilder("simple_7nodes_internetworking")
n.add_qnode([QNodeAddr(1,1), QNodeAddr(3,1)], is_initiator=True)
n.add_qnode([QNodeAddr(3,2)])
n.add_qnode([QNodeAddr(3,3)])
n.add_qnode([QNodeAddr(3,4)])
n.add_qnode([QNodeAddr(3,5)])
n.add_qnode([QNodeAddr(1,99),QNodeAddr(3,99)])
n.add_qnode([QNodeAddr(2,99),QNodeAddr(4,99)])
n.add_qnode([QNodeAddr(4,2)])
n.add_qnode([QNodeAddr(4,3), QNodeAddr(5,1)])
n.add_qnode([QNodeAddr(5,2)])
n.add_qnode([QNodeAddr(5,3)])
n.add_qnode([QNodeAddr(4,4), QNodeAddr(5,99)])
n.add_qnode([QNodeAddr(4,5)])
n.add_qnode([QNodeAddr(2,1), QNodeAddr(4,1)], is_resipient=True)
n.connect_linear(ChannelOption(quantum_channel_distance=5, classical_channel_distance=5))
network = n.network

sim = NativeSimulator("../quisp")
sim.load(network)
await sim.run()
sim.read_result()
sim.df[sim.df['msg_type'] == 'ConnectionSetupResponse']

In [ ]:

count_rulesets_by_responders(sim.df)

In [ ]:
def generate_network(num_nodes: "int", num_subnets: "int") ->"Network":
    assert num_nodes >= 2
    assert num_subnets >= 0
    subnet_size = num_nodes // num_subnets if num_subnets != 0 else 0

    n = NetworkBuilder(f"linear_{num_nodes}nodes_{num_subnets}subnets_internetworking")
    network_addr = 1
    subnet_addr = num_subnets * 2
    for i in range(1, num_nodes+1):
        addr_list = [QNodeAddr(network_addr, i)]
        if subnet_size > 0:
            if i % subnet_size == 1: # new subnet starts
                network_addr += 1
                subnet_addr += 1
                addr_list.append(QNodeAddr(subnet_addr, i))
            elif i % subnet_size == 0: # end of subnets
                network_addr += 1
                addr_list.append(QNodeAddr(subnet_addr, i))
            else: # in subnet, no top level addr
                addr_list = [QNodeAddr(subnet_addr, i)]
        
        n.add_qnode(addr_list, is_initiator=(i == 1), is_resipient=(i == num_nodes))
    n.connect_linear(ChannelOption(quantum_channel_distance=5, classical_channel_distance=5))
    return n.network


In [ ]:
df = sim.df.drop(["qnic_type", "qnic_index", "qubit_index", "application_id", "application_type" , "event_type", "partner_addr"], axis=1)

df = df.astype({"simtime":"float32", "actual_dest_addr": "string", "actual_src_addr":"string"})
df = df[(df["actual_dest_addr"] == df["address"]) | (df["msg_type"] == "ConnectionSetupRequest")]
df.head(23)

In [ ]:
sims= dict()

for num_nodes in [5,10, 15, 20, 25, 30, 35, 40, 45, 50]:
    for num_subnets in [0, 1,2,3,4,5]: #range(0, num_nodes // 3, 4):
        if num_subnets * 3 > num_nodes:
            continue
        network = generate_network(num_nodes=num_nodes, num_subnets=num_subnets)
        key = network.name
        sims[key] = {
            "network_name": key,
            "network": network,
            "num_nodes": num_nodes,
            "num_subnets": num_subnets,
            "error": None
        }
        
for k in sims:
    print(k)

print(len(sims), "networks")

In [ ]:
for key in sims:
    sim = NativeSimulator("../quisp")
    try:
        sim.load(sims[key]["network"])
        sims[key]["ned_dump"]= sim.network.dump()
        sims[key]["ini_dump"]= sim.config.dump()
        await sim.run()
    except RuntimeError:
        sims[key]["error"] = sim.error_messages
    sim.read_result()
    sims[key]["df"] = sim.df.copy()
    sims[key]["num_rulesets_df"] = count_rulesets_by_responders(sim.df)
    print(count_rulesets_by_responders(sim.df))

In [ ]:
df_sims = pd.DataFrame(
    [
        [
            key,
            sims[key]["num_nodes"],
            sims[key]["num_subnets"],
            sims[key]["num_rulesets_df"]["counts"].min(),
            sims[key]["num_rulesets_df"]["counts"].max(),
            sims[key]["num_rulesets_df"]["counts"].mean(),
        ]
        for key in sims
        if sims[key]["error"] is None
    ],
    columns=[
        "key",
        "num_nodes",
        "num_subnets",
        "num_rulesets_min",
        "num_rulesets_max",
        "num_rulesets_mean",
    ],
)
df_sims


In [ ]:
# save the sims obj to pickle
with open("./sims.pickle", "wb") as f:
    import pickle
    pickle.dump(sims, f)

In [ ]:
# load result from pickle file
# check if sims is defined or not.
try:
    sims
except NameError:
    # if sims is not defined, load it from pickle file
    import pickle
    with open("sims.pickle", "rb") as f:
        sims = pickle.load(f)

In [ ]:
px.line(df_sims, x="num_subnets", y="num_rulesets_mean", color="num_nodes")

In [ ]:
px.line(df_sims, x="num_nodes", y="num_rulesets_mean", color="num_subnets")

In [ ]:
print("exist df",[key for key in sims if sims[key].get("df") is None])
print("error:", [key for key in sims if sims[key]["error"]])
sims[list(sims.keys())[2]]
[sims[k] for k in sims]

